# Preprocessing

In [ ]:
import pandas as pd
import re
import os

In [ ]:
import pandas as pd

INPUT_PATH  = r"D:\KULIAH\TA\scrap berita\news_scrapping\prepro_baru\dataset_kotor\news_tempo_hukum_last3y.csv"
OUTPUT_PATH = r"D:\KULIAH\TA\scrap berita\news_scrapping\prepro_baru\dataset_bersih\tempo_bersih.csv"

df = pd.read_csv(INPUT_PATH, dtype=str, encoding="utf-8", low_memory=False)
df.head()

,source,url,released_iso,released_raw,title,highlight,content,tag
0,tempo,https://www.tempo.co/hukum/sistem-pembayaran-b...,2025-10-16T03:00:00+00:00,16 Oktober 2025 | 10.00 WIB,Sistem Pembayaran Bank Jakarta Diduga Diretas ...,Bank Jakarta mengalami peretasan lebih dari se...,BANK Daerah Khusus Ibu Kota (DKI) Jakarta didu...,"bank-jakarta, bank-dki, peretasan, pembobolan,..."
1,tempo,https://www.tempo.co/hukum/celah-kepabeanan-im...,2025-10-15T23:00:00+00:00,16 Oktober 2025 | 06.00 WIB,Jalur Ilegal Garmen Impor,Para penyelundup memanfaatkan celah kepabeanan...,P ETUGAS Bea dan Cukai mendeteksi rencana peny...,"impor-garmen-ilegal, batam, penyelundupan, imp..."
2,tempo,https://www.tempo.co/hukum/kelemahan-sistem-ke...,2025-10-15T23:00:00+00:00,16 Oktober 2025 | 06.00 WIB,Siasat Para Penyelundup Memanfaatkan Celah Kep...,Petugas Bea-Cukai menyita 12 kontainer yang di...,D IREKTORAT Jenderal Bea dan Cukai menerima no...,"impor-garmen-ilegal, batam, penyelundupan-teks..."
3,tempo,https://www.tempo.co/hukum/ammar-zoni-terjerat...,2025-10-15T06:00:00+00:00,15 Oktober 2025 | 13.00 WIB,Mengapa Ammar Zoni Berulang Terjerat Kasus Nar...,Kali ini Ammar Zoni bukan hanya diduga sebagai...,A KTOR Ammar Zoni kembali terjerat kasus narko...,"narapidana, narapidana-narkotika, narkoba, rum..."
4,tempo,https://www.tempo.co/hukum/iklan-gluten-roti-r...,2025-10-24T05:00:00+00:00,24 Oktober 2025 | 12.00 WIB,Hukuman Bagi Produsen Makanan yang Berbohong M...,Iklan kandungan roti Bake n Grind menjadi urus...,"P ADA dasarnya, iklan adalah alat pemasaran pr...","polisi, roti"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        16476 non-null  object
 1   url           16476 non-null  object
 2   released_iso  16476 non-null  object
 3   released_raw  16476 non-null  object
 4   title         16476 non-null  object
 5   highlight     16476 non-null  object
 6   content       16421 non-null  object
 7   tag           16469 non-null  object
dtypes: object(8)
memory usage: 1.0+ MB


Note:  
- Jumlah judul berita dan konten_berita harus sama
- Tag tidak apa berbeda, karena tidak semua memiliki tag
- Atribut yang akan digunakan adalah judul dan konten_berita

## Null Handling

In [ ]:
df['content'].isna().sum()

55

In [ ]:
df = df.dropna(subset=['content'])
df['content'].isna().sum()

0

In [ ]:
df['tag'].isna().sum()

7

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16421 entries, 0 to 16475
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        16421 non-null  object
 1   url           16421 non-null  object
 2   released_iso  16421 non-null  object
 3   released_raw  16421 non-null  object
 4   title         16421 non-null  object
 5   highlight     16421 non-null  object
 6   content       16421 non-null  object
 7   tag           16414 non-null  object
dtypes: object(8)
memory usage: 1.1+ MB


Note:  
- Baris dengan konten_berita null berhasil dihapus
- Jumlah data non-null pada atribut judul = konten berita

## Duplicate

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.duplicated(subset=['title']).sum()

14

In [ ]:
df.duplicated(subset=['url']).sum()

0

In [ ]:
# df[df.duplicated(subset=['url'])]

In [ ]:
# df = df.drop_duplicates(subset=['url'])
# df.shape

Note:
- Ditemukan data duplikat pada atribut judul, tetapi karena tidak ada yg duplikat dari segi url maka dianggap aman

## Normalisasi Tanggal

In [ ]:
# Konversi ISO → datetime
df['released'] = pd.to_datetime(df['released_iso'], errors='coerce')

# Convert timezone UTC → WIB
df['released'] = df['released'].dt.tz_convert('Asia/Jakarta')

# Format ke tampilan biasa
df['released'] = df['released'].dt.strftime('%d/%m/%Y %H:%M')

## Perbaiki Penulisan

In [ ]:
out = pd.DataFrame({
    "source":  df["source"],
    "released": df["released"],
    "title": df["title"],
    "summary": df["highlight"],   # highlight = summary
    "content": df["content"],
    "tag": df["tag"]
})

In [ ]:
import re

# def remove_tempo_location_prefix(text):
#     if not isinstance(text, str):
#         return text

#     return re.sub(
#         r'(?i)^(?:TEMPO\.CO\s*,?\s*)?(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s*,)?\s*-\s*',
#         '',
#         text
#     ).strip()
def strip_tempo_prefix(text):
    if not isinstance(text, str):
        return text

    # 1) Hapus baris mandiri "TEMPO.CO , Kota -"
    text = re.sub(r'(?i)^\s*TEMPO\.CO\s*,?\s*[A-Za-z. ]+?-+\s*$\n?', '', text, flags=re.MULTILINE)

    # 2) Hapus prefix di awal paragraf:
    text = re.sub(r'(?i)^\s*TEMPO\.CO\s*,?\s*[A-Za-z. ]+?-+\s*', '', text).strip()

    return text

def remove_info_prefix(text):
    if not isinstance(text, str):
        return text
    # Hapus awalan seperti "INFO NASIONAL -", "INFO BISNIS -"
    return re.sub(r'^(INFO\s+[A-Z]+\s*-\s*)', '', text, flags=re.IGNORECASE).strip()

def remove_leading_dash(text):
    if not isinstance(text, str):
        return text
    # Hapus "-" atau "—" di awal kalimat
    return re.sub(r'^[\-\—]\s*', '', text).strip()

def fix_first_word_glue_and_caps(text):
    if not isinstance(text, str):
        return text

    # 1) Fix kasus huruf pertama terpisah: "A DA" → "Ada"
    text = re.sub(r'^([A-Za-z])\s+([a-z]+)', lambda m: m.group(1) + m.group(2), text)

    # 2) Fix kasus Tempo: "L EGALISASI" → "LEGALISASI"
    text = re.sub(r'^([A-Z])\s+([A-Z][a-zA-Z]+)', lambda m: m.group(1) + m.group(2).lower(), text)

    # 3) Kapitalisasi awal kalimat
    return text[:1].upper() + text[1:] if text else text

# # untuk kata pertama
# ACRONYMS = {"KPK", "PT", "TNI", "POLRI", "DPR", "RI", "KPU", "BPK", "KPU", "BNN", "Kejari", "Kejagung"}
# def fix_casing(text):
#     def fix_word(word):
#         # Biarkan jika word ada di daftar singkatan
#         if word in ACRONYMS:
#             return word

#         # Jika seluruhnya kapital
#         if word.isupper():
#             # Jika panjangnya ≤ 4 → kemungkinan singkatan → biarkan
#             if len(word) <= 4:
#                 return word
#             # Jika panjang > 4 → ubah jadi Kapital Awal
#             return word.capitalize()

#         return word

#     return re.sub(r"[A-Za-z]+", lambda m: fix_word(m.group(0)), text)

ACRONYMS = {
    "KPK", "PT", "TNI", "POLRI", "DPR", "RI", "KPU", "BPK", "BNN",
    "MA", "MK", "LAN", "LPSK", "KejarI", "Kejagung"
}

def normalize_first_words(text):
    if not isinstance(text, str):
        return text

    text = text.strip()

    # Ambil sampai tanda titik atau spasi pertama
    parts = text.split(" ", 2)

    # Kalau hanya 1 kata → cek kapitalisasi
    if len(parts) == 1:
        w = parts[0]
        if w.isupper() and w not in ACRONYMS and len(w) > 3:
            return w.capitalize()
        return text

    # Kalau kata pertama kapital semua tapi bukan singkatan
    w1 = parts[0]
    if w1.isupper() and w1 not in ACRONYMS:
        if len(w1) > 3:
            parts[0] = w1.capitalize()

    # Tangani 2 kata kapital sekaligus (contoh: AYAH KORBAN..., HERI GUNAWAN...)
    if len(parts) > 1:
        w2 = parts[1]
        if w1.isupper() and w2.isupper():
            if w1 not in ACRONYMS:
                parts[0] = w1.capitalize()
            if w2 not in ACRONYMS:
                parts[1] = w2.capitalize()

    return " ".join(parts)

def drop_stray_repeated_letter(text):
    if not isinstance(text, str):
        return text
    # Hapus huruf tunggal nyasar sebelum spasi, contoh: "Polisi i mengatakan" → "Polisi mengatakan"
    return re.sub(r'\b([a-zA-Z])\s+(?=[a-z])', '', text)

def remove_tempo_boilerplate(text):
    if not isinstance(text, str):
        return text
    patterns = [
        r'Baca juga:.*$',
        r'Ikuti berita.*$',
        r'Dapatkan update.*$',
        r'Klik untuk.*$',
        # Hanya hapus TEMPO.CO jika dia berada di akhir baris atau akhir teks
        r'\bTEMPO\.CO\b\s*$'
    ]
    for p in patterns:
        text = re.sub(p, '', text, flags=re.IGNORECASE | re.MULTILINE)
    return text.strip()

def fix_punct_spacing_strict(text):
    if not isinstance(text, str):
        return text
    # Hilangkan spasi dobel
    text = re.sub(r'\s+', ' ', text)
    # Rapikan spasi sebelum tanda baca
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    return text.strip()

def rapikan_singkatan(text):
    # hapus spasi setelah "(" dan sebelum ")"
    text = re.sub(r'\(\s*([A-Za-z0-9]+)\s*\)', r'(\1)', text)
    return text

def ensure_period(text):
    if not isinstance(text, str):
        return text

    text = text.strip()

    # kalau sudah berakhir . ! ? → biarkan
    if re.search(r'[.!?]$', text):
        return text

    # kalau berakhir dengan tanda baca lain → hapus lalu tambahkan titik
    text = re.sub(r'[\-:;,]+$', '', text).strip()

    return text + '.'

def fix_spacing(text):
    if not isinstance(text, str):
        return text

    # Hilangkan spasi sebelum tanda baca umum
    text = re.sub(r'\s+([,.!?):])', r'\1', text)

    # Hilangkan spasi sebelum tanda kutip penutup
    text = re.sub(r'\s+(["”])', r'\1', text)

    # Pastikan ada spasi setelah kutip kalau diikuti huruf
    text = re.sub(r'(["”])(?=[A-Za-z])', r'\1 ', text)

    # Rapikan spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    return text

In [ ]:
out['content'] = out['content'].apply(strip_tempo_prefix) # awalan tempo
out['content'] = out['content'].apply(remove_info_prefix) # info info di awalan tempo
out['content'] = out['content'].apply(remove_leading_dash) # dash di awal kalimat
out['content'] = out['content'].apply(fix_first_word_glue_and_caps) # normalisasi kata pertama (A DA → Ada)
# out['content'] = out['content'].apply(fix_casing) # normalisasi kata pertama lembaga/bukan
out['content'] = out['content'].apply(normalize_first_words) # normalisasi kata pertama
out['content'] = out['content'].apply(drop_stray_repeated_letter) # huruf lebih (Polisi i → Polisi)

# Hapus Paragraf Promo Iklan Tempo
out['summary'] = out['summary'].apply(remove_tempo_boilerplate)
out['content'] = out['content'].apply(remove_tempo_boilerplate)

# Rapikan Spasi & Tanda Baca
for col in ['title', 'summary', 'content']:
    out[col] = out[col].apply(fix_punct_spacing_strict)

# Rapikan Singkatan dalam kurung
for col in ['title', 'summary', 'content']:
    out[col] = out[col].apply(rapikan_singkatan)

out['content'] = out['content'].apply(lambda x: re.sub(r'\s+', ' ', x).strip()) # Rapikan spasi ganda
out['content'] = out['content'].apply(fix_spacing)
out['title'] = out['title'].apply(ensure_period) # isi titik di akhir judul


### Pengecekan

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

out.sample(10)[['title']]
# out.sample(10)['title'].apply(lambda x: textwrap.fill(x, width=100))

,title
9690,Seorang Mahasiswi Kupang Terseret Kasus Pencabulan Anak yang Libatkan Eks Kapolres Ngada.
11036,Polisi akan Telusuri Berapa Keuntungan Kades Kohod Cs dalam Kasus Pagar Laut Tangerang.
7307,Fakta-Fakta Perusakan Makam Warga Kristen di Bantul.
8966,Empat Respons MA soal Dugaan Suap Hakim Penanganan Korupsi Minyak Goreng.
15337,"Raffi Ahmad Belum Serahkan LHKPN, Ini Sanksi Pejabat yang Tak Laporkan Harta Kekayaan."
8825,Jaksa Kasus Kredit Fiktif BRIguna Buka Peluang Bertambahnya Tersangka.
11316,Polisi Sita Dua Pucuk Senjata Api dan Magasin dari Aske Mabel.
8088,Bisakah Penyidikan Pencucian Uang Mengungkap Asal-usul Uang Zarof Ricar.
9568,"Penembak Jitu untuk Pengamanan Mudik, Ahli Hukum: Jangan Melahirkan Polisi Jagoan."
290,Kejaksaan: Hukum Indonesia Berlaku bagi WNA yang Pimpin BUMN.


In [ ]:
out.sample(5)[['title','summary','content']]

,title,summary,content
6667,Kejagung Bakal Investigasi Pihak yang Loloskan MacBook dan iPad Tom Lembong ke Sel.,Harli menilai yang dilakukan Tom Lembong bisa membuat tahanan lain merasa terdiskriminasi.,"Kejaksaan Agung menegaskan larangan membawa alat elektronik ke dalam kamar tahanan. Kejagung bakal menginvestigasi pihak yang meloloskan MacBook dan iPad ke kamar tahanan terdakwa perkara korupsi impor gula Thomas Trikasih Lembong alias Tom Lembong."" Ketika aturannya menyatakan dilarang ya dilarang. Kami sekarang investigasi siapa yang memasukkan alat komunikasi dan elektronik itu ke kamar yang bersangkutan,"" kata Kepala Pusat Penerangan Hukum Kejagung Harli Siregar di Gedung Kejagung, Jakarta Selatan, Selasa, 3 Juni 2025. Harli menilai yang dilakukan Tom Lembong bisa membuat tahanan lain merasa terdiskriminasi. Pasalnya, kata dia, tahanan lain tidak diizinkan membawa alat elektronik ke kamar tahanan. Menurut Harli, ada beberapa alat elektronik yang bisa dibawa ke dalam rumah tahanan. Akan tetapi tidak dibawa ke dalam kamar."" Televisi itu bisa di luar kamar,"" ujar dia. Sebelumnya, Tom Lembong mengaku kebingungan atas aturan larangan membawa MacBook dan iPad di Rumah Tahanan Negara Salemba. Kendati demikian, dia mengatakan akan bertanggung jawab."" Saya masih sedikit bingung karena ketentuannya melarang benda tajam,"" kata Tom Lembong di Pengadilan Tindak Pidana Korupsi (Tipikor) Jakarta pada Senin, 2 Juni 2025. Selain itu, ujar dia, aturannya juga melarang membawa korek api karena berisiko menimbulkan kebakaran. Eks Menteri Perdagangan itu mengklaim, MacBook dan iPad adalah alat tulis yang ia gunakan untuk menulis pleidoi. Apalagi ia berencana menulis puluhan halaman dalam nota pembeliannya itu. Selain itu, Macbook dan iPad itu ia gunakan untuk membaca berkas perkaranya yang terdiri atas ribuan halaman. Menurut Tom, lebih efisien membaca dokumen tersebut di tablet atau laptop daripada di kertas yang bertumpuk-tumpuk. Sebelumnya, Tom Lembong ketahuan membawa gadget ke dalam penjara. Hal ini terungkap dalam persidangan perkara dugaan korupsi impor gula pada Kamis, 22 Mei 2025. Jaksa penuntut umum mendakwa Tom Lembong merugikan keuangan negara sebesar Rp 578.105.411.622,47 (Rp 578,1 miliar). Angka itu, menurut JPU berdasarkan perhitungan Badan Pengawasan Keuangan dan Pembangunan (BPKP). JPU juga mendakwa Tom memperkaya orang lain atau korporasi sebesar Rp 515.408.740.970,36 (Rp 515,4 miliar). Angka tersebut merupakan bagian dari keuangan negara sebesar Rp 578,1 miliar. Namun, Jaksa dalam surat dakwaannya tidak menjelaskan sisa kerugian Rp 62,7 miliar berasal dari mana. Dinukil dari surat dakwaan Tom Lembong, kerugian keuangan negara sebanyak Rp 578,1 miliar itu berasal dari dua hal. Pertama, dari kemahalan harga yang dibayarkan PT Perusahaan Perdagangan Indonesia (PT PPI) dalam pengadaan gula kristal putih untuk penugasan stabilisasi harga atau operasi pasar. Kedua, dari kekurangan pembayaran bea masuk dan pajak dalam rangka impor (PDRI). Jaksa mendakwa Tom Lembong melanggar Pasal 2 ayat (1) atau Pasal 3 juncto Pasal 18 Undang-Undang tentang Pemberantasan Tindak Pidana Korupsi juncto Pasal 55 ayat (1) ke-1 Kitab Undang-Undang Hukum Pidana (KUHP). Amelia Rahima Sari berkontribusi dalam artikel ini"
86,Empat Polisi Polres Nunukan yang Terlibat Kasus Narkoba hanya Disanksi Etik.,Direktur Tindak Pidana Narkoba Bareskrim Polri mengatakan belum ada proses pidana terhadap keempat polisi itu.,"Empat anggota Kepolisian Resor Nunukan yang diduga terlibat peredaran narkoba masih menjalani proses pemeriksaan etik di Divisi Profesi dan Pengamanan (Propam) Polri. Direktur Tindak Pidana Narkoba Bareskrim Polri Brigadir Jenderal Eko Hadi Santoso mengatakan belum ada proses pidana terhadap keempat polisi itu."" Tindak pidana itu harus terpenuhi unsur-unsur pidananya. Itu sudah terjadi di masa lalu dan pemenuhan barang bukti sudah lewat,"" kata Eko di Gedung Bareskrim Polri, Rabu, 22 Oktober 2025. Kasus itu turut menyeret Kepala Satuan Reserse

In [ ]:
# cek baris spesifik
i = 2728  # ganti nomor baris
print("BEFORE:\n", df.loc[i, 'content'])
print("\nAFTER:\n", out.loc[i, 'content'])

BEFORE:
 KOMISI Pemberantasan Korupsi (KPK) menjaring Wakil Menteri Ketenagakerjaan Immanuel Ebenezer alias Noel bersama dengan 19 orang lainnya dalam operasi tangkap tangan (OTT) di Kementerian Ketenagakerjaan .

Wakil Ketua KPK Fitroh Cahyanto mengatakan, OTT di Kemnaker tersebut dilakukan karena adanya dugaan pemerasan terhadap perusahaan dalam pengurusan sertifikasi Keselamatan dan Kesehatan Kerja (K3). "Pemerasan terhadap perusahaan-perusahaan terkait pengurusan sertifikasi K3," kata Fitroh melalui pesan singkat pada Kamis, 21 Agustus 2025.

OTT KPK yang menjaring Noel menambah daftar panjang skandal di Kemnaker. Setidaknya ada tiga kasus korupsi di Kemnaker. Berikut tiga kasus tersebut:

Korupsi Sistem Proteksi Pekerja Migran Indonesia

Kasus pertama adalah korupsi sistem proteksi pekerja migran atau tenaga kerja Indonesia (TKI). Kasus ini menyeret Reyna Usman sebagai tersangka korupsi pengadaan sistem proteksi TKI 2012 di Kementerian Tenaga Kerja dan Transmigrasi (Kemenakertrans

## Simpan Data Bersih

In [ ]:
out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
print("Saved to:", OUTPUT_PATH)
out.head()

Saved to: D:\KULIAH\TA\scrap berita\news_scrapping\prepro_baru\dataset_bersih\tempo_bersih.csv


source          released  \
0  tempo  16/10/2025 10:00   
1  tempo  16/10/2025 06:00   
2  tempo  16/10/2025 06:00   
3  tempo  15/10/2025 13:00   
4  tempo  24/10/2025 12:00   

                                                                               title  \
0  Sistem Pembayaran Bank Jakarta Diduga Diretas hingga Rp 200 Miliar Lewat BI Fast.   
1                                                         Jalur Ilegal Garmen Impor.   
2                             Siasat Para Penyelundup Memanfaatkan Celah Kepabeanan.   
3                               Mengapa Ammar Zoni Berulang Terjerat Kasus Narkotik.   
4                        Hukuman Bagi Produsen Makanan yang Berbohong Melalui Iklan.   

                                                                                                                 summary  \
0                     Bank Jakarta mengalami peretasan lebih dari sekali. Peretasan terakhir terjadi pada 24 Maret 2025.   
1  Para penyelundup memanfaatkan celah kepabeanan. Hilangnya 12 kontainer barang sitaan dari Batam salah satu contohnya.   
2       Petugas Bea-Cukai menyita 12 kontainer yang diduga berisi barang selundupan. Sempat hilang dari depo penimbunan.   
3                           Kali ini Ammar Zoni bukan hanya diduga sebagai pengguna, tapi juga sebagai pengedar narkoba.   
4                 Iklan kandungan roti Bake n Grind menjadi urusan polisi. Klaim bebas gluten yang tak sesuai kenyataan.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

## Drop Column Not Relevant

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv ('/content/tempo_bersih.csv', engine='python')

In [ ]:
df.head()
df.columns

Index(['source', 'released', 'title', 'summary', 'content', 'tag'], dtype='object')

In [ ]:
df.isnull().sum()

,0
source,0
released,0
title,0
summary,0
content,0
tag,7


In [ ]:
df[['content', 'summary']].isnull().sum()

,0
content,0
summary,0


In [ ]:
print("Jumlah baris content:", df['content'].notnull().sum())
print("Jumlah baris summary:", df['summary'].notnull().sum())


Jumlah baris content: 16421
Jumlah baris summary: 16421


In [ ]:
df["word_count"] = df["content"].astype(str).str.split().str.len()
df["char_count"] = df["content"].astype(str).str.len()
df[["content", "word_count", "char_count"]].head()


,content,word_count,char_count
0,Bank Daerah Khusus Ibu Kota (DKI) Jakarta didu...,603,4366
1,Petugas Bea dan Cukai mendeteksi rencana penye...,293,2214
2,Direktorat Jenderal Bea dan Cukai menerima not...,46,324
3,Aktor Ammar Zoni kembali terjerat kasus narkot...,38,263
4,"Pada dasarnya, iklan adalah alat pemasaran pro...",26,183


In [ ]:
df["word_count"].describe()

,word_count
count,16421.000000
mean,385.738323
std,186.818052
min,13.000000
25%,274.000000
50%,351.000000
75%,466.000000
max,2739.000000


In [ ]:
# Drop berita yang terlalu pendek (anggap belum terscrap)
df_clean = df[df["word_count"] >= 200]

print(f"Jumlah sebelum: {len(df)}, sesudah dibersihkan: {len(df_clean)}")

# df_clean.to_csv("news_tempo_hukum_clean.csv", index=False)

Jumlah sebelum: 16421, sesudah dibersihkan: 15027


In [ ]:
short_news = df[df["word_count"] < 200]
short_news[["title", "word_count"]]


,title,word_count
2,Siasat Para Penyelundup Memanfaatkan Celah Kep...,46
3,Mengapa Ammar Zoni Berulang Terjerat Kasus Nar...,38
4,Hukuman Bagi Produsen Makanan yang Berbohong M...,26
9,Bisakah Sandra Dewi Melawan Sita Aset Korupsi ...,44
12,Ketika Pengemudi Ojek Online Jadi Intelijen Po...,47
...,...,...
16352,Bunuh Diri Anak dan Remaja Naik.,32
16353,Kiat Para Pejabat Menitipkan Calon Pilihan Pim...,74
16354,Siapa Saja Calon Pimpinan KPK yang Direstui Is...,79
16355,Bagaimana Panitia Seleksi Memilih Calon Pimpin...,63


In [ ]:
# === TEXT CLEANING + BEFORE/AFTER REPORT ===
import re, html, unicodedata as ud
import pandas as pd

# Menggunakan df_clean
assert 'df_clean' in globals(), "df_clean belum dibuat. Jalankan cell sebelumnya dulu."
df_clean = df_clean.copy()
df_clean['content'] = df_clean['content'].astype(str)
df_clean['summary'] = df_clean['summary'].astype(str)

# -------- util cleaning ----------
MOJIBAKE_MAP = {
    "â€œ": '"', "â€": '"', "â€˜": "'", "â€™": "'", "â€š": "'", "â€ž": '"',
    "â€“": "-", "â€”": "-", "â€¦": "...", "Â": "", "Â°": "°", "â€Z": "", "â€²": "'", "â€³": '"',
    "â€": "", "â€™": "'", "â€˜": "'", "â„¢": "™", "Ã©": "é", "Ã¨": "è", "Ã": "à",
    "Å": "Š", "œ": "oe", "â€™": "'", "â€Œ": "", "â€Ž": "", "â€ ": '"', "â€": '"',
}

SMART_QUOTES_MAP = {"“": '"', "”": '"', "‘": "'", "’": "'", "—": "-", "–": "-"}

RE_HTML_TAG = re.compile(r"<[^>]+>")
RE_URL = re.compile(r"(https?|ftp)://\S+")
RE_MULTISPACE = re.compile(r"[ \t\u00A0\u2000-\u200B\u202F\u205F\u3000]+")
RE_CONTROL = re.compile(r"[\u0000-\u0008\u000B-\u000C\u000E-\u001F\u007F]")
RE_ZW = re.compile(r"[\u200B-\u200D\uFEFF]")

def normalize_unicode(s: str) -> str:
    return ud.normalize("NFKC", s)

def fix_mojibake(s: str) -> str:
    for bad, good in MOJIBAKE_MAP.items():
        s = s.replace(bad, good)
    for bad, good in SMART_QUOTES_MAP.items():
        s = s.replace(bad, good)
    return s

# def try_latin1_roundtrip(s: str) -> str:
#     # Heuristik: hanya jika terdeteksi pola mojibake
#     if ("â" in s) or ("Â" in s):
#         try:
#             return s.encode("latin1","ignore").decode("utf-8","ignore")
#         except Exception:
#             return s
#     return s
def try_latin1_roundtrip(s: str) -> str:
    try:
        return s.encode("latin1", "ignore").decode("utf-8", "ignore")
    except Exception:
        return s

def strip_html_and_entities(s: str) -> str:
    s = html.unescape(s)
    s = RE_HTML_TAG.sub(" ", s)
    s = RE_URL.sub(" ", s)
    return s

def remove_invisible_and_controls(s: str) -> str:
    s = RE_ZW.sub("", s)
    s = RE_CONTROL.sub(" ", s)
    return s

def normalize_spaces(s: str) -> str:
    s = RE_MULTISPACE.sub(" ", s)
    s = re.sub(r"\s*\n\s*", "\n", s)
    return s.strip()

def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = normalize_unicode(s)
    s = fix_mojibake(s)
    s = try_latin1_roundtrip(s)
    s = fix_mojibake(s)                 # second pass setelah roundtrip
    s = strip_html_and_entities(s)
    s = remove_invisible_and_controls(s)
    s = normalize_spaces(s)
    return s

# def normalize_coordinates(text):
#     # Pulihkan derajat dan simbol prime/double prime
#     text = re.sub(r'(\d{1,3})[\s]*[oO°]?', r'\1°', text)  # derajat
#     text = re.sub(r"(?<=\d)[']{1}(?=\d)", "′", text)       # menit: prime
#     text = re.sub(r"(?<=\d)[\"]{1}(?=\s|\d)", "″", text)   # detik: double prime
#     text = re.sub(r"(?<=\d)\s?(?=[A-Z])", " ", text)       # tambah spasi sebelum S/E
#     return text

# def clean_punctuation_glitches(s):
#     if not isinstance(s, str):
#         return ""
#     # Hapus kombinasi tanda baca berulang
#     s = re.sub(r"([.,!?;:])\1+", r"\1", s)         # "!!" => "!", ".." => "."
#     s = re.sub(r"([.,!?;:])([’”\"'])", r"\1", s)    # titik + kutipan => titik
#     s = re.sub(r"([’”\"'])([.,!?;:])", r"\2", s)    # kutipan + titik => titik
#     s = re.sub(r"([’”\"'])([’”\"'])", r"\1", s)     # kutipan ganda => satu
#     s = re.sub(r"\s+([.,!?;:])", r"\1", s)          # hapus spasi sebelum tanda baca
#     s = re.sub(r"([.,!?;:])([^\s])", r"\1 \2", s)   # jaga spasi setelah tanda baca
#     return s.strip()

# --------- BEFORE/AFTER REPORT ----------
probe_tokens = ["â€", "â€œ","â€\x9d","â€™","â€“","Â","“","”","‘","’","–","—","…"]

def report(df, label):
    print(f"\n=== {label} ===")
    for t in probe_tokens:
        n_c = df['content'].str.contains(re.escape(t), na=False).sum()
        n_s = df['summary'].str.contains(re.escape(t), na=False).sum()
        if n_c or n_s:
            print(f"{repr(t):8s} | content:{n_c:5d} | summary:{n_s:5d}")
    # khusus permintaan: token 'â€'
    k_c = df['content'].str.contains("â€", na=False).sum()
    k_s = df['summary'].str.contains("â€", na=False).sum()
    print(f"[FOCUS 'â€'] content:{k_c} | summary:{k_s}")

# Tampilkan contoh baris yang mengandung 'â€' SEBELUM cleaning
report(df_clean, "SEBELUM CLEANING")
mask_before = df_clean['content'].str.contains("â€", na=False)
if mask_before.any():
    display(df_clean.loc[mask_before, ['content','summary']].head(3))

# --------- APPLY CLEANING ----------
# df_clean["content"] = df_clean["content"].map(clean_text).map(normalize_coordinates)
# df_clean["content"] = df_clean["content"].map(clean_text).map(clean_punctuation_glitches)
df_clean["content"] = df_clean["content"].map(clean_text)
df_clean['summary'] = df_clean['summary'].map(clean_text)


=== SEBELUM CLEANING ===
'“'      | content: 9045 | summary:    0
'”'      | content: 9129 | summary:    0
'‘'      | content:  446 | summary:    0
'’'      | content:  617 | summary:    0
'–'      | content: 1061 | summary:    0
'—'      | content:  303 | summary:    0
'…'      | content:    7 | summary:    0
[FOCUS 'â€'] content:0 | summary:0


In [ ]:
# Tampilkan contoh baris yang tadinya mengandung 'â€' SESUDAH cleaning
report(df_clean, "SESUDAH CLEANING")
mask_after = df_clean['content'].str.contains("â€", na=False)
if mask_after.any():
    display(df_clean.loc[mask_after, ['content','summary']].head(3))
else:
    print("✅ Tidak ada lagi 'â€' di kolom content.")


=== SESUDAH CLEANING ===
[FOCUS 'â€'] content:0 | summary:0
✅ Tidak ada lagi 'â€' di kolom content.


In [ ]:
# Tambahan check setelah cleaning
suspect_tokens = ["â€", "Â", "â€œ", "â€Z", "Â°", "Ã", "Å", "œ"]
for t in suspect_tokens:
    n = df_clean["content"].str.contains(re.escape(t), na=False).sum()
    if n:
        print(f"Masih ada simbol mencurigakan '{t}' di content sebanyak:", n)

In [ ]:
# df_final = pd.read_csv ('/content/news_tempo_hukum_clean.csv', engine='python')
df_clean.to_csv("tempo_hukun_dengan_kolom2.csv", index=False, encoding='utf-8-sig')

In [ ]:
print("Jumlah baris total:", len(df_clean))
print("Jumlah content kosong:", df_clean['content'].isna().sum())
print("Jumlah summary kosong:", df_clean['summary'].isna().sum())

# Drop baris yang tidak punya content atau summary
df_final = df_clean.dropna(subset=['content', 'summary'])

# Sisakan hanya kolom content dan summary
df_final = df_clean[['content', 'summary']]

# Simpan hasil akhir
df_final.to_csv('/content/tempo_final2.csv', index=False, encoding='utf-8-sig')

print("✅ Dataset final tersimpan dengan hanya 2 kolom (content & summary)")
print("Jumlah baris final:", len(df_final))

Jumlah baris total: 15027
Jumlah content kosong: 0
Jumlah summary kosong: 0
✅ Dataset final tersimpan dengan hanya 2 kolom (content & summary)
Jumlah baris final: 15027


In [ ]:
df_final.head()
df_final.columns

Index(['content', 'summary'], dtype='object')

In [ ]:
# Menampilkan 5 baris pertama dataset final
df_final.head()

,content,summary
0,Bank Daerah Khusus Ibu Kota (DKI) Jakarta didu...,Bank Jakarta mengalami peretasan lebih dari se...
1,Petugas Bea dan Cukai mendeteksi rencana penye...,Para penyelundup memanfaatkan celah kepabeanan...
5,Sidang praperadilan Direktur Eksekutif Lokatar...,Sidang praperadilan Direktur Eksekutif Lokatar...
6,"Manajer Marketing PT Indi Daya Group, Muhammad...",Kredit fiktif Bank Jatim merugikan keuangan ne...
7,Kuasa hukum Khariq Anhar telah menyerahkan vid...,Tim hukum Khariq menilai penangkapan kliennya ...


## Data Splitting

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Muat dataset final
df = pd.read_csv('/content/tempo_final2.csv')

# Tampilkan jumlah data awal
print("📊 Total data sebelum split:", len(df))

📊 Total data sebelum split: 15027


In [ ]:
# Bagi menjadi train (80%) dan temp (20%)
train_set, temp_set = train_test_split(df, test_size=0.2, random_state=42)

# Dari temp, bagi lagi menjadi val (10%) dan test (10%)
val_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=42)


In [ ]:
# Tampilkan hasil pembagian
print("\n✅ Data berhasil dibagi:")
print(f"- Data Latih     : {len(train_set)}")
print(f"- Data Validasi  : {len(val_set)}")
print(f"- Data Pengujian : {len(test_set)}")


✅ Data berhasil dibagi:
- Data Latih     : 12021
- Data Validasi  : 1503
- Data Pengujian : 1503


In [ ]:
# Simpan ke file CSV
output_dir = '/content/dataset_split'
os.makedirs(output_dir, exist_ok=True)

train_set.to_csv(os.path.join(output_dir, 'data_latih.csv'), index=False)
val_set.to_csv(os.path.join(output_dir, 'data_validasi.csv'), index=False)
test_set.to_csv(os.path.join(output_dir, 'data_uji.csv'), index=False)
